# Analyzing Receipts with Form Recognizer

<p style='text-align:center'><img src='./images/receipt_analysis.jpg' alt='A robot holding a receipt'/></p>

In the artificial intelligence (AI) field of computer vision, optical character recognition (OCR) is commonly used to read printed or handwritten documents. Often, the text is simply extracted from the documents into a format that can be used for further processing or analysis.

A more advanced OCR scenario is the extraction of information from forms, such as purchase orders or invoices, with a semantic understanding of what the fields in the form represent. The **Form Recognizer** service is specifically designed for this kind of AI problem.

## View a receipt

In this example, you'll use the Form Recognizer's built-in model for analyzing receipts.

Click the green <span style="color:green">&#9655</span> button at the top left of the cell below to run it and see an example of a receipt that you'll use Form Recognizer to analyze.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Create a Form Recognizer resource

Start by creating a Form Recognizer resource in your Azure subscription:

1. In another browser tab, open the Azure portal at https://portal.azure.com, signing in with your Microsoft account.
2. Select **+ Create a resource**, and search for *Form Recognizer*.
3. In the list of services, select **Form Recognizer**.
4. In the **Form Recognizer** blade, select **Create**.
5. In the **Create** blade, enter the following details and select **Create**
  * **Name**: A unique name for your service
  * **Subscription**: Your Azure subscription
  * **Location**: Any available location
  * **Pricing tier**: F0
  * **Resource Group**: The existing resource group you used previously
  * **I confirm I have read and understood the notice below**: Selected.
6. Wait for the service to be created.
7. View your newly created Form Recognizer service in the Azure portal and on the **Quick Start** page, copy the **Key1** and **Endpoint** values and paste them in the code cell below, replacing **YOUR_FORM_KEY** and **YOUR_FORM_ENDPOINT**.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Analyze a receipt

Now you're ready to use Form Recognizer to analyze a receipt.

In [ ]:
from python_code import form_recognizer
import os

try:
    # Read the image data
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
    with open(image_path, "rb") as f:
        data = f.read()
    
    # Send the image data to Form Recognizer
    receipt_data = form_recognizer.get_form_data(form_key, form_endpoint, data)

    # Print the results
    print('\n')
    for field in receipt_data:
        print(field, receipt_data[field])

except Exception as ex:
    print('Error:', ex)

Note that Form Recognizer is able to interpret the data in the form, correctly identifying the merchant address and phone number, and the transaction date and time, as well as the line items, subtotal, tax, and total amounts.

> **Note**: If you're curious about the code used to submit the request to Form Recognizer and process the results, look at the **form_recognizer.py** file in the **python_code** folder.

## More Information

For moer information about the Form Recognizer service, see [the Form Recognizer documentation](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/index)